In [1]:
import numpy as np
import pandas
# import matplotlib.pyplot as plt
import os
import ray
from modin import pandas as pd
from nums import numpy as nps
from nums.core import settings
import nums
import boto3
# from tqdm import tqdm

In [2]:
# Cluster settings
os.environ["RAY_IGNORE_UNHANDLED_ERRORS"] = "0"
# ray.init(ignore_reinit_error=True, address="auto") # On cluster
ray.init() # On local machine
if len(ray.nodes()) > 1:
    settings.cluster_shape = (len(ray.nodes()) - 1, 1)
print(settings.cluster_shape)
nums.init()

(1, 1)
Using driver node ip as head node.
head node 10.0.0.198
total cpus 64.0
device_grid (0, 0) 0=node:10.0.0.198/cpu:1


### Reproduce float get error (resolved)

### from_modin AssertionError for small dataframes (resolved)

### NotFittedError: ElasticNet instance has not been fitted

In [3]:
from nums import from_modin
from nums.sklearn import (
    ElasticNet,
    GradientBoostingRegressor,
    _check_array, instance, #
)

# df = pd.DataFrame(data={"A": np.arange(100), "B": np.arange(1, 101), "C": np.arange(2, 102)}, dtype=float)
# df.at[1, "B"] = np.nan
# display(df.head())
# X = from_modin(df[["A", "B"]])
# y = from_modin(df[["C"]])

melih_size = 10
X, y = nps.random.randn(melih_size, 2), nps.random.randint(2, size=melih_size)
X[1, 1] = np.nan
model = ElasticNet()
model.fit(X, y) # No error because doesn't call ray.get()

# Disassembling model.fit
# _check_array(X, True)
# _check_array(y, True)
# instance().cm.call_actor_method(model.actor, "fit", X.flattened_oids()[0], y.flattened_oids()[0])

model.predict(X[0:2]).get() # Throws NotFittedError with no indication of NaN anywhere
# Reproduced in scripts/script_sklearn.py
# Seems like ray.get() only raises the last error encountered

# model = GradientBoostingRegressor()
# model.fit(X, y)
# model.predict(X[2:4]).get() # Throws NotFittedError, but would throw "Input contains NaN" if arg to predict contains NaN

RayTaskError(NotFittedError): [36mray::ModelActor.predict()[39m (pid=54907, ip=10.0.0.198, repr=<nums.sklearn.ElasticNetActor object at 0x7ff094d4a990>)
  File "/home/ubuntu/anaconda3/envs/aws-asdi/lib/python3.7/site-packages/nums/sklearn.py", line 165, in predict
    return self.instance.predict(X)
  File "/home/ubuntu/anaconda3/envs/aws-asdi/lib/python3.7/site-packages/sklearn/linear_model/_base.py", line 238, in predict
    return self._decision_function(X)
  File "/home/ubuntu/anaconda3/envs/aws-asdi/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py", line 893, in _decision_function
    check_is_fitted(self)
  File "/home/ubuntu/anaconda3/envs/aws-asdi/lib/python3.7/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/home/ubuntu/anaconda3/envs/aws-asdi/lib/python3.7/site-packages/sklearn/utils/validation.py", line 1041, in check_is_fitted
    raise NotFittedError(msg % {'name': type(estimator).__name__})
skl

In [4]:
from sklearn.linear_model import ElasticNet

# df = pd.DataFrame(data={"A": np.arange(100), "B": np.arange(1, 101), "C": np.arange(2, 102)}, dtype=float)
# df.at[1, "B"] = np.nan
# display(df.head())
# X = from_modin(df[["A", "B"]])
# y = from_modin(df[["C"]])

melih_size = 10
X, y = np.random.randn(melih_size, 2), np.random.randint(2, size=melih_size)
X[1, 1] = np.nan

model = ElasticNet()
model.fit(X, y) # Throws ValueError right away (expected)
model.predict(X[0:2])

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').